In [ ]:
!pip install xgboost
!pip install lightgbm

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve, RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, plot_confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import lightgbm as lgb

import os

In [ ]:
ds = pd.read_csv('transaction_dataset.csv', index_col=[0])
ds.drop(columns='Index', inplace=True)

In [ ]:
print(ds.shape)
ds.sample(3)

In [ ]:
ds.drop(columns=[' ERC20 avg time between rec tnx', ' ERC20 avg time between rec 2 tnx', ' ERC20 avg time between contract tnx',
                 ' ERC20 min val sent contract', ' ERC20 max val sent contract', ' ERC20 avg val sent contract', ' ERC20 avg time between sent tnx'], inplace=True)

In [ ]:
missing_values = ds.isna()
missing_percent = missing_values.sum() / ds.shape[0] * 100
missing_df = pd.DataFrame([missing_values.sum(), missing_percent], ['count', 'percent'])
display(missing_df.sort_values(by='percent', axis=1, ascending=False))
missing_df.sort_values(by='percent', axis=1, ascending=False).to_csv('missing.csv')

sns.heatmap(missing_values, cbar=False, cmap='magma')
plt.show()

In [ ]:
non_fraud_rows, fraud_rows = np.where( [ds.iloc[:,0]==1] )
print(ds.iloc[fraud_rows,:].isna().sum()[-20:])

In [ ]:
preprocessing_pipeline = Pipeline([
    ('impoter', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

X = ds.drop(columns='FLAG').to_numpy()
y = ds['FLAG'].to_numpy()

random_permutation = np.random.permutation(len(X))
X = X[random_permutation]
y = y[random_permutation]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.transform(X_test)